In [6]:
!find /usr/local/cuda-* -iname 'libdevice'
!find /usr/local/cuda-* -iname 'libnvvm.so'

/usr/local/cuda-11.0/nvvm/libdevice
/usr/local/cuda-11.0/nvvm/lib64/libnvvm.so


In [7]:
%pip install numba

%pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 26.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 MB 61.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [10]:

from __future__ import annotations

import argparse
import logging

import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
import tensorflow as tf

from numba import cuda, njit

@cuda.jit
def check_gpus(_: None, gpus_optional: bool = False) -> None:
    """Validates that we are detecting GPUs, otherwise raise a RuntimeError."""
    gpu_devices = tf.config.list_physical_devices("GPU")
    if gpu_devices:
        logging.info(f"Using GPU: {gpu_devices}")
    elif gpus_optional:
        logging.warning("No GPUs found, defaulting to CPU.")
    else:
        raise RuntimeError("No GPUs found.")

@cuda.jit
def run(input_text: str, beam_args: list[str] | None = None) -> None:
    beam_options = PipelineOptions(direct_num_workers=0,  # default threads/subprocess to the number of cores on this machine
    direct_running_mode='multi_threading')
    pipeline = beam.Pipeline(InteractiveRunner(), options=beam_options)
    (
        pipeline
        | "Create data" >> beam.Create([input_text])
        | "Check GPU availability"
        >> beam.Map(
            lambda x, unused_side_input: x,
            unused_side_input=beam.pvalue.AsSingleton(
                pipeline | beam.Create([None]) | beam.Map(check_gpus)
            ),
        )
        | "My transform" >> beam.Map(logging.info)
    )
    pipeline.run()

In [ ]:
PROJECT_ID = 'cool-machine-learning'
_REGION = 'us-central1'
_JOB_NAME = 'test1'
_TEMP_LOCATION = 'rw-cool-test'


In [3]:
args = " --runner=DataflowRunner --project=$PROJECT_ID --region=$_REGION --job_name=$_JOB_NAME --temp_location=$_TEMP_LOCATION \
 --machine_type=n1-standard-4 \
 --experiment=worker_accelerator=type:$_GPU_TYPE;count:$_GPU_COUNT;install-nvidia-driver \
 --experiment=use_runner_v2 --experiment=no_use_multiple_sdk_containers --disk_size_gb=50"

In [12]:
logging.getLogger().setLevel(logging.INFO)

run('hello')

ValueError: 
Kernel launch configuration was not specified. Use the syntax:

kernel_function[blockspergrid, threadsperblock](arg0, arg1, ..., argn)

See https://numba.readthedocs.io/en/stable/cuda/kernels.html#kernel-invocation for help.

